<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q gradio pandas

import pandas as pd
import re
import datetime
import gradio as gr
import urllib.parse

# ==============================
# 1. 讀取 Google Sheet
# ==============================

CSV_URL = "https://docs.google.com/spreadsheets/d/1LBjqBkz9YTMhPv_eKtqafvGj0vQesx6hMfV0KvpUVcM/export?format=csv"

def load_data():
    df = pd.read_csv(CSV_URL)
    # 移除店名空白列
    if "店家名稱" in df.columns:
        df = df.dropna(subset=["店家名稱"])
    return df

df_rest = load_data()

# ==============================
# 2. 欄位前處理：價位 / 距離
# ==============================

def parse_price(price_str):
    """從「約100元、100–150」這種字串抓出數字後取平均"""
    if pd.isna(price_str):
        return None
    nums = re.findall(r"\d+", str(price_str))
    if not nums:
        return None
    nums = list(map(int, nums))
    return sum(nums) / len(nums)

df_rest["平均價位"] = df_rest["均價"].apply(parse_price)

def parse_distance(dist_str):
    """從「步行3–5分鐘」「約10~15分鐘」抓分鐘數平均"""
    if pd.isna(dist_str):
        return None
    nums = re.findall(r"\d+", str(dist_str))
    if not nums:
        return None
    nums = list(map(int, nums))
    return sum(nums) / len(nums)

df_rest["預估分鐘"] = df_rest["離師大距離"].apply(parse_distance)

# ==============================
# 3. 營業時間相關處理
# ==============================

weekday_map = {
    "週一": 0,
    "週二": 1,
    "週三": 2,
    "週四": 3,
    "週五": 4,
    "週六": 5,
    "週日": 6,
}
weekday_keywords = {
    0: ["週一", "星期一", "禮拜一"],
    1: ["週二", "星期二", "禮拜二"],
    2: ["週三", "星期三", "禮拜三"],
    3: ["週四", "星期四", "禮拜四"],
    4: ["週五", "星期五", "禮拜五"],
    5: ["週六", "星期六", "禮拜六"],
    6: ["週日", "星期日", "禮拜日"],
}

def normalize_biz_str(biz_str):
    """把營業時間字串全都統一成好處理的格式"""
    s = str(biz_str)
    # 全形冒號 → 半形
    s = s.replace("：", ":")
    # 不同的「到」記號 → 統一用 -
    for ch in ["~", "～", "至"]:
        s = s.replace(ch, "-")
    s = s.replace("–", "-").replace("—", "-")
    return s

def is_closed_today(biz_str, weekday_idx):
    """
    判斷今天是不是公休：
    - 有寫「公休」且有對應的星期關鍵字 → 公休
    - 有寫「不定休」「無固定公休」 → 不視為公休
    """
    s = str(biz_str)
    if "不定休" in s or "無固定公休" in s:
        return False
    if "公休" not in s:
        return False

    for kw in weekday_keywords.get(weekday_idx, []):
        if kw in s:
            return True
    return False

def parse_time_ranges(biz_str):
    """
    解析營業時間字串為多段時間區間 (start_time, end_time) list
    支援：
      11:00–14:00、11:00-14:00、11:00 ~ 14:00、11:00至14:00 等
    自動略過不合法時間（如 25:00），避免程式炸掉
    """
    if pd.isna(biz_str):
        return []

    s = normalize_biz_str(biz_str)

    # 24 小時 or 全日：直接當作 00:00–23:59
    if ("24小時" in s) or ("全日" in s) or ("全天" in s):
        return [(datetime.time(0, 0), datetime.time(23, 59))]

    pattern = r"(\d{1,2}):(\d{2})\s*-\s*(\d{1,2}):(\d{2})"
    ranges = []

    for h1, m1, h2, m2 in re.findall(pattern, s):
        try:
            h1 = int(h1)
            m1 = int(m1)
            h2 = int(h2)
            m2 = int(m2)

            # 處理 24:00 → 當作 23:59
            if h1 == 24 and m1 == 0:
                h1, m1 = 23, 59
            if h2 == 24 and m2 == 0:
                h2, m2 = 23, 59

            # 時間合法性檢查
            if not (0 <= h1 <= 23 and 0 <= m1 <= 59 and 0 <= h2 <= 23 and 0 <= m2 <= 59):
                continue

            start = datetime.time(h1, m1)
            end = datetime.time(h2, m2)
            ranges.append((start, end))
        except Exception:
            continue

    return ranges

df_rest["time_ranges"] = df_rest["營業時間"].apply(parse_time_ranges)

def time_in_range(t, time_range):
    """支援一般時段與跨午夜時段"""
    start, end = time_range
    if start <= end:
        # 例如 11:00–14:00
        return start <= t <= end
    else:
        # 例如 16:00–01:00（跨日）
        return (t >= start) or (t <= end)

def is_open(biz_str, ranges, query_time, weekday_idx):
    """
    判斷該店在指定「星期 + 時間」是否有營業
    無明確時間資料 → 當作有營業（避免把資料不完整的店全砍掉）
    """
    s = str(biz_str)
    # 先看是不是公休
    if is_closed_today(s, weekday_idx):
        return False

    # 沒時間資料，用「保守友善」：視為有營業
    if not ranges:
        return True

    # 任一時段包含查詢時間就算有營業
    for r in ranges:
        if time_in_range(query_time, r):
            return True
    return False

# ==============================
# 4. AI 總結：三家店壓成 100 字說明
# ==============================

def ai_summary(df3):
    """
    把最多三家店的優缺點壓成約 100 字，幫助快速決策
    規則：
      - 價格：<=100 超平價 / <=200 價格親民 / >200 稍偏貴
      - 距離：<=5 很近 / <=10 步行可達 / >10 略遠
      - 加上簡短「販售內容＋特色」關鍵字
    """
    lines = []
    for _, r in df3.iterrows():
        name = str(r.get("店家名稱", "")).strip()
        price = r.get("平均價位", None)
        dist = r.get("預估分鐘", None)
        cate = str(r.get("販售內容", "")).strip()
        feat = str(r.get("特色", "")).strip()

        pros = []
        cons = []

        # 價格判斷
        if price is not None:
            if price <= 100:
                pros.append("超平價")
            elif price <= 200:
                pros.append("價格親民")
            else:
                cons.append("稍偏貴")

        # 距離判斷
        if dist is not None:
            if dist <= 5:
                pros.append("離師大很近")
            elif dist <= 10:
                pros.append("步行可達")
            else:
                cons.append("距離略遠")

        if cate:
            pros.append(cate[:8])
        if feat:
            pros.append(feat[:8])

        desc = f"{name}："
        if pros:
            desc += "優點：" + "、".join(pros)
        if cons:
            desc += "；缺點：" + "、".join(cons)
        lines.append(desc)

    text = " / ".join(lines)
    # 粗略限制在 100 個字元以內
    if len(text) > 100:
        text = text[:100] + "..."
    return text

# ==============================
# 5. 地圖 HTML：顯示 3 家店的 Google Maps
# ==============================

def build_maps_html(df3):
    """
    用地點欄位組出 Google Maps iframe，直接在 Gradio 裡顯示地圖
    """
    blocks = []
    for _, r in df3.iterrows():
        name = str(r.get("店家名稱", "")).strip()
        addr = str(r.get("地點", "")).strip()
        if not addr:
            continue
        query = urllib.parse.quote_plus(addr)
        # 這裡不用 API key，單純用搜尋 + embed
        url = f"https://www.google.com/maps?q={query}&output=embed"

        html = (
            "<div style='margin-bottom:20px;'>"
            f"<div><b>{name}</b> - {addr}</div>"
            f"<iframe src='{url}' width='100%' height='260' "
            "style='border:0;' loading='lazy' "
            "referrerpolicy='no-referrer-when-downgrade'></iframe>"
            "</div>"
        )
        blocks.append(html)

    if not blocks:
        return "<p>沒有可顯示的地圖。</p>"
    return "".join(blocks)

# ==============================
# 6. 主決策函式
# ==============================

def recommend(budget, max_minutes, time_str, weekday_label):
    if not time_str:
        return "請輸入時間，例如 12:30", "", pd.DataFrame(), "<p>尚未產生地圖。</p>"

    # 解析時間字串
    try:
        h, m = map(int, time_str.split(":"))
        qtime = datetime.time(h, m)
    except Exception:
        return "時間格式錯誤，請輸入 HH:MM，例如 12:30", "", pd.DataFrame(), "<p>時間格式錯誤，無法產生地圖。</p>"

    weekday_idx = weekday_map.get(weekday_label, None)
    if weekday_idx is None:
        return "請選擇星期幾", "", pd.DataFrame(), "<p>尚未產生地圖。</p>"

    df = df_rest.copy()

    # 價位篩選
    df = df[df["平均價位"].notna()]
    df = df[df["平均價位"] <= budget]

    # 距離篩選
    df = df[df["預估分鐘"].notna()]
    df = df[df["預估分鐘"] <= max_minutes]

    # 營業時間篩選
    def check_open(row):
        return is_open(row["營業時間"], row["time_ranges"], qtime, weekday_idx)

    df = df[df.apply(check_open, axis=1)]

    if df.empty:
        return (
            "找不到符合條件的店家，可以試著放寬預算或距離，或檢查是否為公休日。",
            "",
            pd.DataFrame(),
            "<p>沒有符合條件的店家，因此無地圖可顯示。</p>",
        )

    # 依距離優先，再依價位排序，取前 3 家
    df = df.sort_values(by=["預估分鐘", "平均價位"])
    top3 = df.head(3)

    show_cols = ["店家名稱", "販售內容", "地點", "特色", "均價", "離師大距離", "營業時間"]
    result = top3[show_cols].reset_index(drop=True)

    summary = ai_summary(top3)
    maps_html = build_maps_html(top3)

    main_msg = f"幫你挑出 {len(result)} 家候選午餐（依距離優先、再看價位）："
    ai_msg = f"AI 快速決策建議：{summary}"

    return main_msg, ai_msg, result, maps_html

# ==============================
# 7. Gradio 介面
# ==============================

weekdays = list(weekday_map.keys())

with gr.Blocks() as demo:
    gr.Markdown(
        "## 師大附近午餐決策器 🍱\n"
        "輸入預算、可接受距離與用餐時間，幫你挑出 3 家目前有開的餐廳，"
        "並附上 AI 優缺點摘要與地圖預覽。"
    )

    with gr.Row():
        budget = gr.Slider(
            50, 1000, step=50, value=200,
            label="預算上限 (NT$，依平均價位計算)"
        )
        max_minutes = gr.Slider(
            3, 30, step=1, value=10,
            label="可接受距離 (預估分鐘)"
        )

    with gr.Row():
        time_str = gr.Textbox(
            label="用餐時間 (24小時制，例如 12:30)",
            value="12:30"
        )
        weekday_label = gr.Dropdown(
            choices=weekdays,
            value="週一",
            label="星期幾"
        )

    btn = gr.Button("幫我決定午餐！")

    msg_out = gr.Markdown(label="篩選結果")
    ai_out = gr.Markdown(label="AI 快速決策建議")
    table_out = gr.Dataframe(label="推薦 3 家", interactive=False)
    map_out = gr.HTML()

    btn.click(
        fn=recommend,
        inputs=[budget, max_minutes, time_str, weekday_label],
        outputs=[msg_out, ai_out, table_out, map_out]
    )

demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>